# EDA on 2010 Medicare Beneficiary Summary

This notebook explores cost and chronic condition patterns in synthetic Medicare data from 2010. The goal is to identify trends and potential high-impact patient profiles.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


## Load 2010 Beneficiary Data

In [ ]:
df = pd.read_csv("../data/raw/synpuf_sample1/DE1_0_2010_Beneficiary_Summary_File_Sample_1.csv")


## Calculate Total Reimbursement Cost
Sum inpatient, outpatient, and carrier reimbursements into a single column.


In [ ]:
df["TOTAL_COST"] = df["MEDREIMB_IP"] + df["MEDREIMB_OP"] + df["MEDREIMB_CAR"]


## Distribution of Total Cost per Patient


In [ ]:
plt.hist(df["TOTAL_COST"], bins=50)
plt.title("Total Reimbursement Cost per Patient")
plt.xlabel("Cost ($)")
plt.ylabel("Count")
plt.show()


## Filter Patients in the Top 10% of Total Cost


In [ ]:
high_cost_cutoff = df["TOTAL_COST"].quantile(0.9)
high_cost_df = df[df["TOTAL_COST"] >= high_cost_cutoff]
print(f"High-cost patients: {len(high_cost_df)}")


## Count Chronic Conditions per Patient
Each chronic condition is a binary flag. We sum them to get a total burden score.


In [ ]:
chronic_cols = [col for col in df.columns if col.startswith("SP_")]
df_chronic = df[["DESYNPUF_ID", "TOTAL_COST"] + chronic_cols]
df_chronic["CHRONIC_COUNT"] = df_chronic[chronic_cols].sum(axis=1)


## Scatter Plot: Chronic Conditions vs. Total Cost


In [ ]:
plt.scatter(df_chronic["CHRONIC_COUNT"], df_chronic["TOTAL_COST"], alpha=0.5)
plt.xlabel("Number of Chronic Conditions")
plt.ylabel("Total Cost")
plt.title("Cost vs. Chronic Conditions")
plt.show()


## Create Heuristic-Based Scoring
A simple score to surface interesting patient profiles based on cost and chronic indicators.


In [ ]:
def heuristic_score(row):
    score = 0
    if row["TOTAL_COST"] > high_cost_cutoff:
        score += 2
    if row["CHRONIC_COUNT"] >= 3:
        score += 2
    if row["SP_DIABETES"] == 1 or row["SP_CHF"] == 1:
        score += 1
    return score

df_chronic["SCORE"] = df_chronic.apply(heuristic_score, axis=1)


## Preview Top-Scoring Profiles


In [ ]:
df_candidates = df_chronic[df_chronic["SCORE"] >= 4]
print(f"Flagged rows: {len(df_candidates)}")
df_candidates.head()


## Save Selected Profiles


In [ ]:
df_candidates.to_csv("../data/ai_opportunity_candidates.csv", index=False)
